In [1]:
import pickle
import random
import logging
import time
import pickle
import json
import os
import pathlib
from os.path import dirname, abspath
from rdkit import Chem

import numpy as np
import torch
from rdkit import RDLogger

import rootutils
root_dir = rootutils.setup_root(".",
                               indicator=".project-root",
                               pythonpath=True)

In [2]:
# load /home/ubuntu/recursiveLLM/data/routes_possible_test_hard.pkl
with open('/home/ubuntu/recursiveLLM/data/routes_possible_test_hard.pkl', 'rb') as f:
    routes = pickle.load(f)
    # print the keys of the dictionary

In [3]:
routes[1]

['CCCC[C@@H](C(=O)N1CCC[C@H]1C(=O)O)[C@@H](F)C(=O)OC>>CCCC[C@@H](C(=O)N1CCC[C@H]1C(=O)OC(C)(C)C)[C@@H](F)C(=O)OC',
 'CCCC[C@@H](C(=O)N1CCC[C@H]1C(=O)OC(C)(C)C)[C@@H](F)C(=O)OC>>O=S(=O)(OS(=O)(=O)C(F)(F)F)C(F)(F)F.CCCC[C@@H](C(=O)N1CCC[C@H]1C(=O)OC(C)(C)C)[C@H](O)C(=O)OC',
 'CCCC[C@@H](C(=O)N1CCC[C@H]1C(=O)OC(C)(C)C)[C@H](O)C(=O)OC>>CCCCC(C(=O)N1CCC[C@H]1C(=O)OC(C)(C)C)C1OC(OC)(c2ccccc2)OC1=O',
 'CCCCC(C(=O)N1CCC[C@H]1C(=O)OC(C)(C)C)C1OC(OC)(c2ccccc2)OC1=O>>CCCCC(C(=O)O)C1OC(OC)(c2ccccc2)OC1=O.CC(C)(C)OC(=O)[C@@H]1CCCN1',
 'CCCCC(C(=O)O)C1OC(OC)(c2ccccc2)OC1=O>>Cc1ccccc1.CCCC[C@@H](C(=O)O)[C@H](O)C(=O)O.CC1(C)C2CCC1(CS(=O)(=O)O)C(=O)C2',
 'CCCC[C@@H](C(=O)O)[C@H](O)C(=O)O>>CCCC[C@@H](C(=O)OC)[C@H](O)C(=O)OC',
 'CCCC[C@@H](C(=O)OC)[C@H](O)C(=O)OC>>CC=CC[C@@H](C(=O)OC)[C@H](O)C(=O)OC',
 'CC=CC[C@@H](C(=O)OC)[C@H](O)C(=O)OC>>COC(=O)CC(O)C(=O)OC.[Li]CCCC']

In [4]:
target_mols = [Chem.CanonSmiles(route[0].split('>')[0]) for route in routes]
num_targets = len(target_mols)
target_mols_id = list(range(num_targets))
max_len = 16 # for USPTO test molecules

In [5]:
len(target_mols)

190

In [6]:
random.seed(21)
# sample 10 molecules from target_mols with seed 21
sampled_mols = random.sample(target_mols, 10)
# print the sampled molecules
# save sampled_mols to csv file
import pandas as pd
df = pd.DataFrame(sampled_mols, columns=['smiles'])
df.to_csv(f"{root_dir}/data/USPTO_190_sample.csv", index=False)


In [7]:
sampled_mols

['Cn1nccc1[C@]1(O)CCCC[C@H]1O',
 'CC1=NC2(N=C1N)c1cc(-c3cc(Cl)cc(C#N)c3)ccc1CCC21CC1',
 'CCC/C=C/[C@H]1CC[C@H](C(CO)CO)CC1',
 'C=CCC1C=C(C)CC(C)CC(OC)C2OC(O)(C(=O)C(=O)N3CCCCC3C(=O)OC(C(C)=CC3CCC(O)C(OC)C3)C(C)C=CC1O)C(C)CC2OC',
 'CC(=O)NC[C@H]1CC[C@@H](C(=O)Nc2cccc(OC(F)(F)F)c2)N1',
 'COC(=O)c1ccc2c(c1)C=CC(=C(Cl)Cl)CO2',
 'Cc1ccc(C(C)(C)C)cc1S[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@H]1O',
 'COC(=O)[C@@H]1CCCC2(CCCCC2)[C@H]1O',
 'CC(C)S(=O)(=O)N[C@H]1CCOC[C@H]1c1ccc(I)cc1',
 'CN1CC[C@@H](c2c(O)cc(O)c3c(=O)cc(-c4ccc(C(F)(F)F)cc4)oc23)[C@@H]1CO']